In [1]:
import os
import gzip
from struct import Struct
import pandas as pd
import time

In [2]:
matrix_path = './alevin_matrices/alevin/'
output_path = './alevin_output/'

def get_cb_count():
    cb_count = dict()
    for dir_name in os.listdir(matrix_path):
        if dir_name[0] != '.':
            f = open(matrix_path + '/' + dir_name + '/quants_mat_rows.txt', 'r')
            for line in f:
                line = line.strip()
                if line in cb_count:
                    cb_count[line] = cb_count[line] + 1
                else:
                    cb_count[line] = 1
            f.close()
    return cb_count

cb_count = get_cb_count()

In [ ]:
def merge():
    start = time.time()
    values_file = open(output_path + '/concatenated.txt', 'w')
    rows_file = open(output_path + '/concatenated_rows.txt', 'w')

    common_cb = dict()

    for dir_name in os.listdir(matrix_path):
        if dir_name[0] != '.':

            gene_file = matrix_path + dir_name + '/quants_mat_cols.txt'
            print('gene file', gene_file)
            gene_names = pd.read_table(gene_file, header=None)[0].values
            num_genes = len(gene_names)
            header_struct = Struct( "d" * num_genes)

            rows = open(matrix_path + '/' + dir_name + '/quants_mat_rows.txt', 'r')
            matrix = gzip.open(matrix_path + '/' + dir_name + '/quants_mat.gz', 'r')
            count = 0
            while True:
                count += 1
                try:
                    cell_values = header_struct.unpack_from(matrix.read(header_struct.size))
                except:
                    print('counted: ', count)
                    print('Error in unpack')
                    break
                cb = next(rows).strip()
                if cb_count[cb] == 1:
                    values_file.write(str(cell_values) + '\n')
                    rows_file.write(cb + '\n')
                else:
                    if cb in common_cb:
                        common_cb[cb].append(cell_values)
                    else:
                        common_cb[cb] = [cell_values]
            rows.close()
            matrix.close()

    for key, val in common_cb.items():
            index = 0
            for cell_values in val:
                rows_file.write(key + '_' + str(index) + '\n')
                values_file.write(str(cell_values) + '\n')
                index += 1

    values_file.close()
    rows_file.close()
    end = time.time()
    print('Finished. Time: ', end - start)
    
merge()

gene file ./alevin_matrices/alevin/mouse_2/quants_mat_cols.txt
